In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
import numpy as np

In [ ]:
def get_data(inputPath, fileName, numberOfFiles, columnNames):
    files = []
    for i in range(0, numberOfFiles):
        files.append(pd.read_csv("{}{}({}).txt".format(inputPath, fileName, i)))
        files[i] = files[i][['step'] + columnNames]
    data = files[0]
    for i in range(1, numberOfFiles):
        data = data.merge(files[i], on = 'step', suffixes=(str(i-1), str(i)))
    return data

def smooth(steps, values, steps_new):
    spl = make_interp_spline(steps, values, k=1) #BSpline object
    return spl(steps_new)

In [ ]:
# paths
inputPath = './output/'
outputPath = './metrics/'
fileName = '1D_advanced_Sequential10000_BoltzmannQ_10000steps'
numberOfFiles = 8

In [ ]:
data = get_data(inputPath, fileName, numberOfFiles, ['loss', 'mean_q']).drop(columns=['step'])
data

In [ ]:
data = data.drop(range(0,11))

In [ ]:
data.reset_index(inplace=True, drop=True)

In [ ]:
data.to_csv(fileName + '.csv')

# Model

In [ ]:
def regression(x, y, polynom = 30, sample = 100):
    z = np.polyfit(x, y, polynom)
    f = np.poly1d(z)
    x_new = np.linspace(x[0], x[-1], sample)
    y_new = f(x_new)
    return x_new, y_new

In [ ]:
def plot_interval_confidence(data, min_values, max_values):
    x, y_1 = regression(data.index, min_values)
    plt.plot(x, y_1, label = 'borne_min', color = 'grey')
    x, y_2 = regression(data.index, max_values)
    plt.plot(x, y_2, label = 'borne_max', color = 'grey')
    plt.fill_between(x, y_1, y_2, color = 'grey', alpha = 0.1)

In [ ]:
def plot_metric(data, min_max, metric_name):
    plt.figure(figsize = (20,10))
    for (min_values, max_values) in min_max :
        plot_interval_confidence(data, min_values, max_values)
    for i in range(numberOfFiles):
        x, y = regression(data.index, data[metric_name+str(i)])
        plt.plot(x, y, alpha = .6, label = i)

## Loss

In [ ]:
losses = pd.DataFrame(data['loss0'])
for i in range(1,8):
    losses['loss'+str(i)] = data['loss'+str(i)]
losses

In [ ]:
means = losses.T.mean()
stds = losses.T.std()
min_values_68 = means - stds/(numberOfFiles)**(.5)
max_values_68 = means + stds/(numberOfFiles)**(.5)

min_values_95 = means - 2 * stds/(numberOfFiles)**(.5)
max_values_95 = means + 2 * stds/(numberOfFiles)**(.5)

min_values_99 = means - 3 * stds/(numberOfFiles)**(.5)
max_values_99 = means + 3 * stds/(numberOfFiles)**(.5)

min_max = [(min_values_68, max_values_68), (min_values_95, max_values_95), (min_values_99, max_values_99)]

In [ ]:
plot_metric(data, min_max, 'loss')
plt.xlabel('Episode')
plt.ylabel('Perte')
plt.title("Courbe de perte pour l'agent avancé paramètres A")
plt.savefig('LOL.png')

### Mean_Q

In [ ]:
mean_q = pd.DataFrame(data['mean_q0'])
for i in range(1,8):
    mean_q['loss'+str(i)] = data['mean_q'+str(i)]
mean_q

In [ ]:
means = mean_q.T.mean()
stds = mean_q.T.std()
min_values_68 = means - stds/(numberOfFiles)**(.5)
max_values_68 = means + stds/(numberOfFiles)**(.5)

min_values_95 = means - 2 * stds/(numberOfFiles)**(.5)
max_values_95 = means + 2 * stds/(numberOfFiles)**(.5)

min_values_99 = means - 3 * stds/(numberOfFiles)**(.5)
max_values_99 = means + 3 * stds/(numberOfFiles)**(.5)

min_max = [(min_values_68, max_values_68), (min_values_95, max_values_95), (min_values_99, max_values_99)]

In [ ]:
plot_metric(data, min_max, 'mean_q')
plt.xlabel('Episode')
plt.ylabel('Q-moyenne')
plt.title("Courbe de Q-moyenne pour l'agent avancé paramètres A")
plt.savefig('LOL.png')